## Introduction to Creating RAGs (Retrieval-Augmented Generators) with OpenAI

## 1. - Langchan LLM Chain Tutorial

## 1.1 - Build a basic agent

In [4]:
from langchain.agents import create_agent
import os
import getpass

os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY")

In [ ]:
import os

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    tools=[get_weather],
    system_prompt="You are a helpful assistant",

)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='7773e436-2b62-4f36-833a-e391d28dd98f'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "sf"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c5e4a-a846-7d13-9b7a-ca262030c509-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': '2846b2f6-f579-4b94-a2de-1ea5e9c0ac4f', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 51, 'output_tokens': 15, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in sf!", name='get_weather', id='b4dd15e2-fa7e-4ad6-b472-715ab72293b3', tool_call_id='2846b2f6-f579-4b94-a2de-1ea5e9c0ac4f'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_n

## 1.2 - Build a real-world agent

In [16]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. 
If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

In [8]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city. """
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema. """
    user_id: str


@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

In [12]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "google_genai:gemini-2.5-flash-lite",
    temperature=0.5,
    timeout=10,
    max_tokens=10000
)

In [13]:
@dataclass 
class ResponseFormat:
    """Response schema for the agent."""
    punny_response: str
    weather_conditions: str | None = None

In [14]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [17]:
from langchain.agents.structured_output import ToolStrategy 

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_weather_for_location, get_user_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])

response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you"}]},
    config = config,
    context = Context(user_id="1")
)

print(response['structured_response'])

ResponseFormat(punny_response='I can\'t give you a "weather" report, but I can tell you it\'s always sunny in Florida!', weather_conditions='Sunny')
ResponseFormat(punny_response='You\'re welcome! I hope that was "weather" you were looking for!', weather_conditions=None)


## 2. Build a RAG agent with LangChain

In [ ]:
model = init_chat_model("google_genai:gemini-2.5-flash-lite")

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model = "models/gemini-embedding-001")

In [2]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os

pc = Pinecone(os.environ.get("PINECONE_API_KEY"))
index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

NameError: name 'index_name' is not defined

In [ ]:
import ba4 
from langchain_